<a href="https://colab.research.google.com/github/karthiksuresh8/ML_project/blob/main/Multilingual_Customer_Support_Tickets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary libraries

In [ ]:

#!pip install datasets

!pip install pyarrow==13.0.0

import pandas as pd
import plotly.express as px
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import torch
import numpy as np

In [ ]:
# Load dataset
data_path = "ticket_helpdesk_labeled_multi_languages_english_spain_french_german.csv"
df = pd.read_csv(data_path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Display the first few rows of the dataset
print("First few rows of the dataset:")
print(df.head())

First few rows of the dataset:
        queue  priority software_used   hardware_used  \
0    Hardware         2           NaN  Wireless Mouse   
1    Hardware         2           NaN          IP PBX   
2    Hardware         2           NaN    SFX-Netzteil   
3  Accounting         2           NaN             NaN   
4    Software         2      Arbitrum             NaN   

                     accounting_category language  \
0                                    NaN       en   
1                                    NaN       fr   
2                                    NaN       de   
3  Customer Inquiries::Technical Support       en   
4                                    NaN       en   

                                 subject  \
0  Wireless Mouse suddenly stops working   
1          Problème de connexions IP PBX   
2        Problem mit meinem SFX-Netzteil   
3             Invoice Adjustment Request   
4    Issue with Arbitrum: UI not loading   

                                          

In [ ]:
# EDA
print("\nEDA and Visualization")
print("\nSummary statistics:")
print(df.describe(include='all'))


EDA and Visualization

Summary statistics:
           queue    priority      software_used hardware_used  \
count        200  200.000000                 83            66   
unique         3         NaN                 81            63   
top     Software         NaN  Sales Forecasting        IP PBX   
freq          83         NaN                  3             2   
mean         NaN    2.195000                NaN           NaN   
std          NaN    0.781041                NaN           NaN   
min          NaN    1.000000                NaN           NaN   
25%          NaN    2.000000                NaN           NaN   
50%          NaN    2.000000                NaN           NaN   
75%          NaN    3.000000                NaN           NaN   
max          NaN    3.000000                NaN           NaN   

                                   accounting_category language  \
count                                               51      200   
unique                                   

In [ ]:
# Plot the distribution of the 'queue' column
fig_queue = px.histogram(df, x='queue', title='Distribution of Queue Categories', color='queue')
fig_queue.show()


In [ ]:
# Plot the distribution of the 'priority' column
fig_priority = px.histogram(df, x='priority', title='Distribution of Priority Levels', color='priority')
fig_priority.show()

In [ ]:
fig_language = px.histogram(df, x='language', title='Distribution of Languages', color='language')
fig_language.show()

In [ ]:
fig_software = px.histogram(df, x='software_used', title='Distribution of Software Used', color='software_used')
fig_software.show()

In [ ]:
fig_hardware = px.histogram(df, x='hardware_used', title='Distribution of Hardware Used', color='hardware_used')
fig_hardware.show()

In [ ]:
fig_accounting = px.histogram(df, x='accounting_category', title='Distribution of Accounting Categories', color='accounting_category')
fig_accounting.show()

In [ ]:
# Subplot for distribution of 'priority', 'language', and 'queue'
fig = make_subplots(rows=3, cols=1, subplot_titles=('Priority Distribution', 'Language Distribution', 'Queue Distribution'))

fig.add_trace(go.Histogram(x=df['priority'], name='Priority'), row=1, col=1)
fig.add_trace(go.Histogram(x=df['language'], name='Language'), row=2, col=1)
fig.add_trace(go.Histogram(x=df['queue'], name='Queue'), row=3, col=1)

fig.update_layout(title_text='Distributions of Priority, Language, and Queue', showlegend=False)
fig.show()

In [ ]:
# 'priority' vs. 'queue'
fig_scatter = px.scatter(df, x='priority', y='queue', color='priority', title='Scatter Plot of Priority vs. Queue')
fig_scatter.show()

In [ ]:
# Encode target variable (queue)
df['queue_encoded'] = df['queue'].astype('category').cat.codes
queue_mapping = dict(enumerate(df['queue'].astype('category').cat.categories))

In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['queue_encoded'], test_size=0.2, random_state=42)

In [ ]:
# Create a Dataset object for HuggingFace
train_data = Dataset.from_dict({'text': X_train.tolist(), 'label': y_train.tolist()})
test_data = Dataset.from_dict({'text': X_test.tolist(), 'label': y_test.tolist()})

In [ ]:
# Data Preprocessing
df = df.dropna(subset=['text'])  # Drop rows where 'text' is NaN
df['text'] = df['text'].astype(str)

In [ ]:
# Load pre-trained tokenizer and model
model_name = "xlm-roberta-base"  # Multilingual model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=df['queue_encoded'].nunique())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize the data
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)

train_data = train_data.map(preprocess_function, batched=True)
test_data = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="/content/output/",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning:

`evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead



In [ ]:
# Compute metrics for evaluation
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        'accuracy': (preds == p.label_ids).astype(np.float32).mean().item(),
    }

In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.057322,0.400000
2,No log,0.375333,0.900000
3,No log,0.401319,0.875000
4,No log,0.590482,0.825000
5,No log,0.347951,0.925000
6,No log,0.453653,0.925000
7,No log,0.408963,0.925000
8,No log,0.384953,0.925000
9,No log,0.352993,0.925000
10,No log,0.347297,0.925000


TrainOutput(global_step=200, training_loss=0.2329764747619629, metrics={'train_runtime': 2532.6793, 'train_samples_per_second': 0.632, 'train_steps_per_second': 0.079, 'total_flos': 83045172470400.0, 'train_loss': 0.2329764747619629, 'epoch': 10.0})

In [ ]:
# Evaluate the model
trainer.evaluate()

{'eval_loss': 0.34729695320129395,
 'eval_accuracy': 0.925000011920929,
 'eval_runtime': 12.4836,
 'eval_samples_per_second': 3.204,
 'eval_steps_per_second': 0.401,
 'epoch': 10.0}

In [ ]:
# Predictions
predictions = trainer.predict(test_data)
preds = torch.tensor(predictions.predictions).argmax(axis=1)

# Show classification report
print(classification_report(y_test, preds, target_names=[queue_mapping[i] for i in range(len(queue_mapping))]))

              precision    recall  f1-score   support

  Accounting       1.00      0.67      0.80         9
    Hardware       0.88      1.00      0.93        14
    Software       0.94      1.00      0.97        17

    accuracy                           0.93        40
   macro avg       0.94      0.89      0.90        40
weighted avg       0.93      0.93      0.92        40



In [ ]:
# Show confusion matrix
cm = confusion_matrix(y_test, preds)
fig_cm = px.imshow(cm, text_auto=True, labels=dict(x="Predicted", y="Actual"), x=list(queue_mapping.values()), y=list(queue_mapping.values()))
fig_cm.update_layout(title="Confusion Matrix")
fig_cm.show()

In [ ]:
# Sample predictions with proper alignment
sample_df = pd.DataFrame({'text': X_test, 'actual': y_test, 'predicted': preds})
sample_df['actual_queue'] = sample_df['actual'].map(queue_mapping)
sample_df['predicted_queue'] = sample_df['predicted'].map(queue_mapping)
sample_df = sample_df.sample(5, random_state=42)

# Beautiful presentation of sample predictions
fig_samples = go.Figure(data=[go.Table(
    header=dict(values=['<b>Text</b>', '<b>Actual Queue</b>', '<b>Predicted Queue</b>'],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[sample_df['text'], sample_df['actual_queue'], sample_df['predicted_queue']],
               fill_color='lavender',
               align='left'))
])

fig_samples.update_layout(title='Sample Predictions')
fig_samples.show()

# Define output path
output_path = "/content/output/"

# Create the directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# Save the output
sample_df.to_csv(output_path + "sample_predictions.csv", index=False)

print(f"Sample predictions saved to {output_path}sample_predictions.csv")

Sample predictions saved to /content/output/sample_predictions.csv


In [ ]:


# Assuming X_test, y_test, preds, and queue_mapping are already defined

# Create a DataFrame with text, actual, and predicted values
full_df = pd.DataFrame({'text': X_test, 'actual': y_test, 'predicted': preds})

# Map the actual and predicted values to their corresponding queues
full_df['actual_queue'] = full_df['actual'].map(queue_mapping)
full_df['predicted_queue'] = full_df['predicted'].map(queue_mapping)

# Beautiful presentation of original predictions
fig_full = go.Figure(data=[go.Table(
    header=dict(values=['<b>Text</b>', '<b>Actual Queue</b>', '<b>Predicted Queue</b>'],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[full_df['text'], full_df['actual_queue'], full_df['predicted_queue']],
               fill_color='lavender',
               align='left'))
])

fig_full.update_layout(title='Original Predictions')
fig_full.show()

# Define output path for Google Colab
output_path = "/content/output/"

# Create the directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# Save the output
full_df.to_csv(output_path + "original_predictions.csv", index=False)

print(f"Original predictions saved to {output_path}original_predictions.csv")


Original predictions saved to /content/output/original_predictions.csv


# New section